In [1]:
import os
import random
import json
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import soundfile as sf
from abc import ABC, abstractmethod
from scipy.ndimage import gaussian_filter
from matplotlib.path import Path
from tqdm import tqdm  

# =========================================================
# 1) SpectrogramModifier
# =========================================================
class SpectrogramModifier:
    def __init__(
        self,
        sample_rate=16000,
        n_fft=1024,
        hop_length=512,
        noise_strength=0.1,
        noise_type='normal',
        noise_params=None
    ):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.noise_strength = noise_strength
        self.noise_type = noise_type
        self.noise_params = noise_params if noise_params else {}
        self.signal = None
        self.signal_with_noise = None
        self.S_db = None

    def _generate_normal_noise(self, length, params):
        mean = params.get('mean', 0.0)
        std = params.get('std', 1.0)
        return np.random.normal(mean, std, length)

    def _generate_uniform_noise(self, length, params):
        low = params.get('low', -1.0)
        high = params.get('high', 1.0)
        return np.random.uniform(low, high, length)

    def _generate_perlin_noise(self, length, params):
        def fade(t):
            return 6 * t**5 - 15 * t**4 + 10 * t**3

        seed = params.get('seed', 42)
        np.random.seed(seed)
        perm = np.arange(256)
        np.random.shuffle(perm)
        perm = np.stack([perm, perm]).flatten()
        scale = params.get('scale', 50.0)
        xs = np.linspace(0, length / scale, length)
        xi = np.floor(xs).astype(int)
        xf = xs - xi
        xi = xi % 256
        left_hash = perm[xi]
        right_hash = perm[xi + 1]
        u = fade(xf)
        left_grad = ((left_hash & 1) * 2 - 1) * xf
        right_grad = ((right_hash & 1) * 2 - 1) * (xf - 1)
        noise = (1 - u) * left_grad + u * right_grad
        noise = noise / np.max(np.abs(noise))
        return noise

    def generate_noise(self, signal):
        length = len(signal)
        noise_type = self.noise_type
        params = self.noise_params
        if noise_type == 'normal':
            noise = self._generate_normal_noise(length, params)
        elif noise_type == 'uniform':
            noise = self._generate_uniform_noise(length, params)
        elif noise_type == 'perlin':
            noise = self._generate_perlin_noise(length, params)
        else:
            noise = np.zeros_like(signal)
        return signal + noise * self.noise_strength

    def compute_spectrogram(self, signal):
        self.signal = signal
        self.signal_with_noise = self.generate_noise(signal)
        S = np.abs(
            librosa.stft(
                self.signal_with_noise,
                n_fft=self.n_fft,
                hop_length=self.hop_length,
                window='hann'
            )
        )
        self.S_db = librosa.amplitude_to_db(S, ref=np.max)
        return self.S_db

    def _get_freqs(self):
        return np.linspace(0, self.sample_rate / 2, self.S_db.shape[0])

    def _get_times(self):
        return librosa.frames_to_time(
            np.arange(self.S_db.shape[1]),
            sr=self.sample_rate,
            hop_length=self.hop_length
        )

    def apply_dB_mask(self, dB_mask):
        self.S_db += dB_mask

    def plot_spectrogram(self, show_labels=True, colormap='magma', title='Spectrogram'):
        if self.S_db is None:
            raise ValueError("compute_spectrogram() must be called first.")
        fig, ax = plt.subplots(figsize=(12, 6))
        img = librosa.display.specshow(
            self.S_db,
            sr=self.sample_rate,
            hop_length=self.hop_length,
            x_axis='time',
            y_axis='linear',
            ax=ax,
            cmap=colormap
        )
        if show_labels:
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Frequency (Hz)')
            ax.set_title(title)
            fig.colorbar(img, ax=ax, format="%+2.0f dB")
        else:
            ax.axis('off')
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=0.5)
        return fig, ax

# =========================================================
# 2) DBShape (Abstract Base Class)
# =========================================================
class DBShape(ABC):
    @abstractmethod
    def create_mask(self, spectro_shape, spectro_mod):
        pass

# =========================================================
# 3) Shape Hierarchy
# =========================================================
class BaseShape(DBShape):
    def __init__(self):
        pass

    @abstractmethod
    def generate_shape_mask(self, spectro_shape, spectro_mod):
        pass

    def create_mask(self, spectro_shape, spectro_mod):
        return self.generate_shape_mask(spectro_shape, spectro_mod)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__dict__})"

class CircleDBShape(BaseShape):
    def __init__(self, center_freq, center_time, radius_freq, radius_time, strength_dB):
        super().__init__()
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = (
            (ff - self.center_freq) ** 2 / (self.radius_freq ** 2)
            + (tt - self.center_time) ** 2 / (self.radius_time ** 2)
        )
        circle = (dist <= 1).astype(float)
        return circle * self.strength_dB

class TrapezoidDBShape(BaseShape):
    def __init__(
        self,
        freq_min,
        freq_max,
        time_min,
        time_max,
        slope_freq,
        slope_time,
        strength_dB
    ):
        super().__init__()
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.slope_freq = slope_freq
        self.slope_time = slope_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()

        freq_mask = (freqs >= self.freq_min) & (freqs <= self.freq_max)
        time_mask = (times >= self.time_min) & (times <= self.time_max)
        f_inds = np.where(freq_mask)[0]
        t_inds = np.where(time_mask)[0]

        if len(f_inds) == 0 or len(t_inds) == 0:
            return mask

        f_dist = (freqs[f_inds] - self.freq_min) / (self.freq_max - self.freq_min)
        t_dist = (times[t_inds] - self.time_min) / (self.time_max - self.time_min)

        for i, fi in enumerate(f_inds):
            for j, ti in enumerate(t_inds):
                val = self.strength_dB
                val *= (1 - abs(f_dist[i] - 0.5) * 2 * self.slope_freq)
                val *= (1 - abs(t_dist[j] - 0.5) * 2 * self.slope_time)
                mask[fi, ti] += val
        return mask

class SpikeDBShape(BaseShape):
    def __init__(
        self,
        center_freq,
        center_time,
        radius_freq,
        radius_time,
        strength_dB,
        rotate=0.0
    ):
        super().__init__()
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB
        self.rotate_deg = rotate

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')

        f_shift = ff - self.center_freq
        t_shift = tt - self.center_time

        angle_rad = np.deg2rad(self.rotate_deg)
        cos_a = np.cos(angle_rad)
        sin_a = np.sin(angle_rad)

        f_rot = f_shift * cos_a - t_shift * sin_a
        t_rot = f_shift * sin_a + t_shift * cos_a

        dist = np.sqrt(
            (f_rot**2) / (self.radius_freq**2) +
            (t_rot**2) / (self.radius_time**2)
        )
        spike = np.exp(-dist**2) * self.strength_dB
        return spike

class PillarDBShape(BaseShape):
    def __init__(self, freq_min, freq_max, strength_dB):
        super().__init__()
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_indices = np.where(
            (freqs >= self.freq_min) & (freqs <= self.freq_max)
        )[0]
        if len(freq_indices) == 0:
            return mask
        mask[freq_indices, :] += self.strength_dB
        return mask

class RectangleDBShape(BaseShape):
    def __init__(
        self,
        freq_min,
        freq_max,
        time_min,
        time_max,
        strength_dB
    ):
        super().__init__()
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_indices = np.where(
            (freqs >= self.freq_min) & (freqs <= self.freq_max)
        )[0]
        time_indices = np.where(
            (times >= self.time_min) & (times <= self.time_max)
        )[0]
        if len(freq_indices) == 0 or len(time_indices) == 0:
            return mask
        mask[np.ix_(freq_indices, time_indices)] += self.strength_dB
        return mask

class EllipseDBShape(BaseShape):
    def __init__(
        self,
        center_freq,
        center_time,
        radius_freq,
        radius_time,
        strength_dB
    ):
        super().__init__()
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = (
            (ff - self.center_freq) ** 2 / (self.radius_freq ** 2) +
            (tt - self.center_time) ** 2 / (self.radius_time ** 2)
        )
        ellipse = (dist <= 1).astype(float)
        return ellipse * self.strength_dB

class HorizontalSpikeDBShape(BaseShape):
    def __init__(
        self,
        center_freq,
        center_time,
        radius_freq,
        radius_time,
        strength_dB
    ):
        super().__init__()
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        spike_shape = SpikeDBShape(
            center_freq=self.center_freq,
            center_time=self.center_time,
            radius_freq=self.radius_freq,
            radius_time=self.radius_time,
            strength_dB=self.strength_dB,
            rotate=0.0
        )
        return spike_shape.generate_shape_mask(spectro_shape, spectro_mod)

class VerticalSpikeDBShape(BaseShape):
    def __init__(
        self,
        center_freq,
        center_time,
        radius_freq,
        radius_time,
        strength_dB
    ):
        super().__init__()
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        spike_shape = SpikeDBShape(
            center_freq=self.center_freq,
            center_time=self.center_time,
            radius_freq=self.radius_freq,
            radius_time=self.radius_time,
            strength_dB=self.strength_dB,
            rotate=0.0 
        )
        return spike_shape.generate_shape_mask(spectro_shape, spectro_mod)

class HorizontalRangeDistributionDBShape(BaseShape):
    def __init__(
        self,
        freq_min,
        freq_max,
        strength_dB,
        distribution='gaussian',
        distribution_params=None
    ):
        super().__init__()
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.strength_dB = strength_dB
        self.distribution = distribution
        self.distribution_params = distribution_params if distribution_params else {}

    def _get_distribution(self, values):
        sigma = self.distribution_params.get('sigma', 1000)
        center = (values.min() + values.max()) / 2
        dist = np.exp(-0.5 * ((values - center) / sigma) ** 2)
        if dist.max() != 0:
            dist /= dist.max()
        return dist

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_indices = np.where(
            (freqs >= self.freq_min) & (freqs <= self.freq_max)
        )[0]

        if len(freq_indices) == 0:
            return mask

        freq_values = freqs[freq_indices]
        dist_values = self._get_distribution(freq_values)

        for i, fi in enumerate(freq_indices):
            mask[fi, :] += self.strength_dB * dist_values[i]

        return mask

class VerticalRangeDistributionDBShape(BaseShape):
    def __init__(
        self,
        time_min,
        time_max,
        strength_dB,
        distribution='gaussian',
        distribution_params=None
    ):
        super().__init__()
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB
        self.distribution = distribution
        self.distribution_params = distribution_params if distribution_params else {}

    def _get_distribution(self, values):
        sigma = self.distribution_params.get('sigma', 0.5)
        center = (values.min() + values.max()) / 2
        dist = np.exp(-0.5 * ((values - center) / sigma) ** 2)
        dist /= dist.max()
        return dist

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_indices = np.where(
            (times >= self.time_min) & (times <= self.time_max)
        )[0]
        if len(time_indices) == 0:
            return mask

        time_vals = times[time_indices]
        dist_values = self._get_distribution(time_vals)

        for i, ti in enumerate(time_indices):
            mask[:, ti] += self.strength_dB * dist_values[i]

        return mask

class HillDBShape(BaseShape):
    def __init__(
        self,
        freq_center,
        time_center,
        freq_width,
        time_width,
        strength_dB
    ):
        super().__init__()
        self.freq_center = freq_center
        self.time_center = time_center
        self.freq_width = freq_width
        self.time_width = time_width
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = np.sqrt(
            ((ff - self.freq_center) ** 2) / (self.freq_width ** 2) +
            ((tt - self.time_center) ** 2) / (self.time_width ** 2)
        )
        hill = (1 - dist)
        hill[hill < 0] = 0
        return hill * self.strength_dB

class FogDBShape(BaseShape):
    def __init__(self, strength_dB, coverage=1.0):
        super().__init__()
        self.strength_dB = strength_dB
        self.coverage = coverage

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        random_map = np.random.uniform(0, 1, spectro_shape)
        fog = (random_map < self.coverage).astype(float)
        fog *= self.strength_dB * (np.random.randn(*spectro_shape) * 0.1)
        return fog

class PolygonDBShape(BaseShape):
    def __init__(self, vertices, strength_dB):
        super().__init__()
        self.vertices = vertices
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')

        path = Path(self.vertices)
        points = np.vstack((ff.ravel(), tt.ravel())).T
        inside = path.contains_points(points).reshape(ff.shape)
        mask = np.zeros(spectro_shape)
        mask[inside] = self.strength_dB
        return mask

class WavePatternDBShape(BaseShape):
    def __init__(self, axis='time', frequency=1.0, strength_dB=5.0):
        super().__init__()
        self.axis = axis
        self.frequency = frequency
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        mask = np.zeros(spectro_shape)
        if self.axis == 'time':
            wave = np.sin(2 * np.pi * self.frequency * times)
            mask += wave[np.newaxis, :] * self.strength_dB
        else:
            wave = np.sin(2 * np.pi * self.frequency * freqs)
            mask += wave[:, np.newaxis] * self.strength_dB
        return mask

class RealWorldNoiseDBShape(BaseShape):
    def __init__(
        self,
        audio_path,
        freq_min,
        freq_max,
        time_min,
        time_max,
        strength_dB=0
    ):
        super().__init__()
        self.audio_path = audio_path
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        y, sr = librosa.load(self.audio_path, sr=spectro_mod.sample_rate)
        S = np.abs(
            librosa.stft(y, n_fft=spectro_mod.n_fft, hop_length=spectro_mod.hop_length)
        )
        S_db = librosa.amplitude_to_db(S, ref=np.max)

        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_indices = np.where(
            (freqs >= self.freq_min) & (freqs <= self.freq_max)
        )[0]
        time_indices = np.where(
            (times >= self.time_min) & (times <= self.time_max)
        )[0]

        if len(freq_indices) == 0 or len(time_indices) == 0:
            return np.zeros(spectro_shape)

        f_cut = min(len(freq_indices), S_db.shape[0])
        t_cut = min(len(time_indices), S_db.shape[1])

        mask = np.zeros(spectro_shape)
        mask[freq_indices[:f_cut], time_indices[:t_cut]] += (
            S_db[:f_cut, :t_cut] + self.strength_dB
        )
        return mask

class HorizontalLineDBShape(BaseShape):
    def __init__(self, center_freq, strength_dB, thickness=2):
        super().__init__()
        self.center_freq = center_freq
        self.strength_dB = strength_dB
        self.thickness = thickness

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_idx = np.argmin(np.abs(freqs - self.center_freq))
        start_idx = max(freq_idx - self.thickness // 2, 0)
        end_idx = min(freq_idx + self.thickness // 2 + 1, spectro_shape[0])
        mask[start_idx:end_idx, :] += self.strength_dB
        return mask

class VerticalLineDBShape(BaseShape):
    def __init__(self, center_time, strength_dB, thickness=2):
        super().__init__()
        self.center_time = center_time
        self.strength_dB = strength_dB
        self.thickness = thickness

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_idx = np.argmin(np.abs(times - self.center_time))
        start_idx = max(time_idx - self.thickness // 2, 0)
        end_idx = min(time_idx + self.thickness // 2 + 1, spectro_shape[1])
        mask[:, start_idx:end_idx] += self.strength_dB
        return mask

# =========================================================
# 4) Pattern Hierarchy
# =========================================================
class Pattern(ABC):
    @abstractmethod
    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        pass

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__dict__})"

class LinearPattern(Pattern):
    def __init__(
        self,
        shape_name,
        shape_params,
        direction='time',
        repeat=5,
        spacing=1.0
    ):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.direction = direction
        self.repeat = repeat
        self.spacing = spacing

    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        mask = np.zeros(spectro_shape)
        for i in range(self.repeat):
            offset = i * self.spacing
            params = self.shape_params.copy()
            if self.direction == 'time' and 'center_time' in params:
                params['center_time'] += offset
            elif self.direction == 'freq' and 'center_freq' in params:
                params['center_freq'] += offset

            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj is not None:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

class RandomPattern(Pattern):
    def __init__(
        self,
        shape_name,
        shape_params,
        n=10,
        freq_range=None,
        time_range=None
    ):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.n = n
        self.freq_range = freq_range if freq_range else (0, 8000)
        self.time_range = time_range if time_range else (0, 10)

    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        mask = np.zeros(spectro_shape)
        for _ in range(self.n):
            params = self.shape_params.copy()
            if 'center_freq' in params:
                params['center_freq'] = np.random.uniform(*self.freq_range)
            if 'center_time' in params:
                params['center_time'] = np.random.uniform(*self.time_range)

            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

class NLinearRepeatTSleepPattern(Pattern):
    def __init__(
        self,
        shape_name,
        shape_params,
        repeat=3,
        repeat_time=0.5,
        repeat_hz=None,
        sleep_time=5.0,
        start_time=0.0,
        direction='time'
    ):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.repeat = repeat
        self.repeat_time = repeat_time
        self.repeat_hz = repeat_hz if repeat_hz is not None else repeat_time * 1000
        self.sleep_time = sleep_time
        self.start_time = start_time
        self.direction = direction

    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        mask = np.zeros(spectro_shape)
        if self.direction == 'time':
            current_time = self.start_time
        elif self.direction == 'freq':
            current_freq = self.start_time
        else:
            raise ValueError("Invalid direction. Use 'time' or 'freq'.")

        for _ in range(self.repeat):
            params = self.shape_params.copy()
            if self.direction == 'time' and 'center_time' in params:
                params['center_time'] = current_time
            elif self.direction == 'freq' and 'center_freq' in params:
                params['center_freq'] = current_freq

            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)

            if self.direction == 'time':
                current_time += self.repeat_time
            elif self.direction == 'freq':
                current_freq += self.repeat_hz

        return mask

class ConvexPattern(Pattern):
    def __init__(
        self,
        shape_name,
        shape_params,
        freq_min,
        freq_max,
        time_min,
        time_max,
        n=10
    ):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.n = n

    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        mask = np.zeros(spectro_shape)
        freqs = np.linspace(self.freq_min, self.freq_max, self.n)
        times = np.linspace(self.time_min, self.time_max, self.n)
        for i in range(self.n):
            params = self.shape_params.copy()
            if 'center_freq' in params:
                params['center_freq'] = freqs[i]
            if 'center_time' in params:
                params['center_time'] = times[i]

            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

class FunctionPattern(Pattern):
    def __init__(self, func):
        self.func = func

    def create_mask(self, spectro_shape, spectro_mod, shape_factory):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        return self.func(ff, tt)

# =========================================================
# 5) Factories (ShapeFactory, PatternFactory)
# =========================================================
class ShapeFactory:
    def create(self, shape_name, **kwargs):
        shape_name = shape_name.lower()
        if shape_name == "circle":
            return CircleDBShape(
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "trapezoid":
            return TrapezoidDBShape(
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                slope_freq=kwargs['slope_freq'],
                slope_time=kwargs['slope_time'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "rectangle":
            return RectangleDBShape(
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "ellipse":
            return EllipseDBShape(
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "fog":
            return FogDBShape(
                strength_dB=kwargs['strength_dB'],
                coverage=kwargs.get('coverage', 1.0)
            )
        elif shape_name == "polygon":
            return PolygonDBShape(
                vertices=kwargs['vertices'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "wave_pattern":
            return WavePatternDBShape(
                axis=kwargs.get('axis', 'time'),
                frequency=kwargs['frequency'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "real_world_noise":
            return RealWorldNoiseDBShape(
                audio_path=kwargs['audio_path'],
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "horizontal_spike":
            return HorizontalSpikeDBShape(
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "vertical_spike":
            return VerticalSpikeDBShape(
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "hill":
            return HillDBShape(
                freq_center=kwargs['freq_center'],
                time_center=kwargs['time_center'],
                freq_width=kwargs['freq_width'],
                time_width=kwargs['time_width'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "pillar":
            return PillarDBShape(
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                strength_dB=kwargs['strength_dB']
            )
        elif shape_name == "horizontal_line":
            return HorizontalLineDBShape(
                center_freq=kwargs['center_freq'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs.get('thickness', 1)
            )
        elif shape_name == "vertical_line":
            return VerticalLineDBShape(
                center_time=kwargs['center_time'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs.get('thickness', 1)
            )
        elif shape_name == "horizontal_range_dist_db":
            return HorizontalRangeDistributionDBShape(
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                strength_dB=kwargs['strength_dB'],
                distribution=kwargs.get('distribution', 'gaussian'),
                distribution_params=kwargs.get('distribution_params', {})
            )
        elif shape_name == "vertical_range_dist_db":
            return VerticalRangeDistributionDBShape(
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB'],
                distribution=kwargs.get('distribution', 'gaussian'),
                distribution_params=kwargs.get('distribution_params', {})
            )
        else:
            raise ValueError(f"Unknown shape name: {shape_name}")

class PatternFactory:
    def create(self, pattern_name, params):
        pattern_name = pattern_name.lower()
        if pattern_name == "linear":
            return LinearPattern(
                shape_name=params['shape_name'],
                shape_params=params['shape_params'],
                direction=params.get('direction', 'time'),
                repeat=params.get('repeat', 5),
                spacing=params.get('spacing', 1.0)
            )
        elif pattern_name == "random":
            return RandomPattern(
                shape_name=params['shape_name'],
                shape_params=params['shape_params'],
                n=params.get('n', 10),
                freq_range=params.get('freq_range', (0, 16000)),
                time_range=params.get('time_range', (0.0, 12.0))
            )
        elif pattern_name == "n_linear_repeat_t_sleep":
            direction = params.get('direction', 'time')
            if direction == 'freq':
                repeat_hz = params.get('repeat_time', 0.5) * 1000
            else:
                repeat_hz = None

            return NLinearRepeatTSleepPattern(
                shape_name=params['shape_name'],
                shape_params=params['shape_params'],
                repeat=params.get('repeat', 3),
                repeat_time=params.get('repeat_time', 0.5),
                repeat_hz=repeat_hz,
                sleep_time=params.get('sleep_time', 5.0),
                start_time=params.get('start_time', 0.0),
                direction=direction
            )
        elif pattern_name == "convex":
            return ConvexPattern(
                shape_name=params['shape_name'],
                shape_params=params['shape_params'],
                freq_min=params['freq_min'],
                freq_max=params['freq_max'],
                time_min=params['time_min'],
                time_max=params['time_max'],
                n=params.get('n', 10)
            )
        elif pattern_name == "function":
            return FunctionPattern(params['func'])
        else:
            return None

# =========================================================
# 6) NoisePipeline
# =========================================================
class NoisePipeline:
    def __init__(
        self,
        spectro_mod,
        apply_blur=False,
        blur_sigma=1.0
    ):
        self.spectro_mod = spectro_mod
        self.shape_factory = ShapeFactory()
        self.pattern_factory = PatternFactory()
        self.shapes = []
        self.patterns = []
        self.apply_blur = apply_blur
        self.blur_sigma = blur_sigma

    def add_shape(self, shape):
        self.shapes.append(shape)
        return self

    def add_pattern(self, pattern):
        if pattern is not None:
            self.patterns.append(pattern)
        return self

    def generate(self, signal):
        spec = self.spectro_mod.compute_spectrogram(signal)
        total_mask = np.zeros_like(spec)

        for shape in self.shapes:
            shape_mask = shape.create_mask(spec.shape, self.spectro_mod)
            total_mask += shape_mask

        for pattern in self.patterns:
            pattern_mask = pattern.create_mask(
                spec.shape,
                self.spectro_mod,
                self.shape_factory
            )
            total_mask += pattern_mask

        if self.apply_blur and self.blur_sigma > 0:
            total_mask = gaussian_filter(total_mask, sigma=self.blur_sigma)

        self.spectro_mod.apply_dB_mask(total_mask)
        return self.spectro_mod.S_db

# =========================================================
# 7) Weighted Item Selection Function
# =========================================================
def pick_item_from_ratio(ratio_dict):
    items = list(ratio_dict.keys())
    weights = list(ratio_dict.values())
    total = sum(weights)
    if total == 0:
        raise ValueError("Total weight cannot be zero.")
    r = random.uniform(0, total)
    s = 0
    for item, w in zip(items, weights):
        s += w
        if r <= s:
            return item
    return items[-1]

# =========================================================
# 8) Generate Random Shape Parameters
# =========================================================
def generate_shape_params(
    shape_name,
    duration,
    sr,
    freq_min=20,
    time_min=0.0,
    min_float_value=0.001,
    alpha=1.0,
    max_db_power=20,
    min_db_power=10
):
    """
    Generates all parameters randomly based on the shape_name.
    Clamps time and frequency values within their specified boundaries.
    """
    params = {
        "strength_dB": random.uniform(min_db_power, max_db_power)
    }

    # Generate frequency bounds
    random_freq_min = random.uniform(freq_min, sr)
    random_freq_max = random.uniform(freq_min, sr)
    if random_freq_min > random_freq_max:
        random_freq_min, random_freq_max = random_freq_max, random_freq_min

    # Generate time bounds
    random_time_min = random.uniform(time_min, duration)
    random_time_max = random.uniform(time_min, duration)
    if random_time_min > random_time_max:
        random_time_min, random_time_max = random_time_max, random_time_min

    # Define a helper function to clamp values
    def clamp(value, min_value, max_value):
        return max(min_value, min(value, max_value))

    # Shape-specific parameter generation
    if shape_name == "horizontal_spike":
        params.update({
            "center_freq": clamp(
                random.uniform(random_freq_min, random_freq_max),
                0,
                sr
            ),
            "center_time": clamp(
                random.uniform(random_time_min, random_time_max),
                0,
                duration
            ),
            "radius_freq": clamp(
                random.uniform(30.0, 100.0),
                15.0,
                sr / 2
            ),
            "radius_time": clamp(
                random.uniform(random_time_max * 0.001, random_time_max * 0.1),
                0.05,
                duration * alpha
            )
        })
    elif shape_name == "vertical_spike":
        params.update({
            "center_freq": clamp(
                random.uniform(random_freq_min, random_freq_max) * 0.001,
                0,
                sr
            ),
            "center_time": clamp(
                random.uniform(random_time_min, random_time_max),
                0,
                duration
            ),
            "radius_freq": clamp(
                random.uniform(random_freq_max * 0.1, random_freq_max),
                0.1 * sr,
                sr
            ),
            "radius_time": clamp(
                random.uniform(random_time_max * 0.01, random_time_max * 0.1),
                0.05,
                duration * alpha
            )
        })
    elif shape_name == "trapezoid":
        params.update({
            "freq_min": clamp(random_freq_min, freq_min, sr),
            "freq_max": clamp(random_freq_max, freq_min, sr),
            "time_min": clamp(random_time_min, time_min, duration),
            "time_max": clamp(random_time_max, time_min, duration),
            "slope_freq": random.uniform(0.0, 2.0),
            "slope_time": random.uniform(0.0, 2.0)
        })
    elif shape_name == "fog":
        params.update({
            "coverage": clamp(random.uniform(0, 1), 0, 1)
        })
    elif shape_name == "hill":
        freq_center = random.uniform(random_freq_min, random_freq_max)
        center_time = random.uniform(random_time_min, random_time_max)
        freq_width = random.uniform(min_float_value, (random_freq_max - random_freq_min) * alpha)
        time_width = random.uniform(min_float_value, duration * alpha)

        params.update({
            "freq_center": clamp(freq_center, 0, sr),
            "time_center": clamp(center_time, 0, duration),
            "freq_width": clamp(freq_width, min_float_value, sr),
            "time_width": clamp(time_width, min_float_value, duration)
        })
    elif shape_name == "wave_pattern":
        params.update({
            "axis": random.choice(["time", "freq"]),
            "frequency": clamp(random.uniform(0.1, 10), 0.1, 10)
        })
    elif shape_name == "polygon":
        num_vertices = random.randint(3, 8)
        vertices = []
        for _ in range(num_vertices):
            f_rand = random.uniform(random_freq_min, random_freq_max)
            t_rand = random.uniform(random_time_min, random_time_max)
            f_rand = clamp(f_rand, 0, sr)
            t_rand = clamp(t_rand, 0, duration)
            vertices.append([f_rand, t_rand])
        params.update({
            "vertices": vertices
        })
    elif shape_name == "rectangle":
        params.update({
            "freq_min": clamp(random_freq_min, freq_min, sr),
            "freq_max": clamp(random_freq_max, freq_min, sr),
            "time_min": clamp(random_time_min, time_min, duration),
            "time_max": clamp(random_time_max, time_min, duration)
        })
    elif shape_name == "pillar":
        params.update({
            "freq_min": clamp(random_freq_min, freq_min, sr),
            "freq_max": clamp(random_freq_max, freq_min, sr)
        })
    elif shape_name == "horizontal_line":
        center_freq = random.uniform(random_freq_min, random_freq_max)
        params.update({
            "center_freq": clamp(center_freq, 0, sr),
            "thickness": random.randint(1, 3)
        })
    elif shape_name == "vertical_line":
        center_time = random.uniform(random_time_min, random_time_max)
        params.update({
            "center_time": clamp(center_time, 0, duration),
            "thickness": random.randint(1, 3)
        })
    elif shape_name == "horizontal_range_dist_db":
        dist_options = ["gaussian"]
        chosen_dist = random.choice(dist_options)
        freq_min_range = clamp(
            random_freq_min - 0.1,  # Adjusted threshold
            0,
            sr
        )
        freq_max_range = clamp(
            random_freq_min + 0.1,  # Adjusted threshold
            0,
            sr
        )
        sigma = clamp(
            random.uniform(1000.0 * alpha, 1100.0 * alpha),
            1000.0 * alpha,
            1100.0 * alpha
        )

        params.update({
            "freq_min": freq_min_range,
            "freq_max": freq_max_range,
            "distribution": chosen_dist,
            "distribution_params": {"sigma": sigma}
        })
    elif shape_name == "vertical_range_dist_db":
        dist_options = ["gaussian"]
        chosen_dist = random.choice(dist_options)
        time_min_range = clamp(
            random_time_min - 0.1,  # Adjusted threshold
            0,
            duration
        )
        time_max_range = clamp(
            random_time_max + 0.1,  # Adjusted threshold
            0,
            duration
        )
        sigma = clamp(
            random.uniform(1.0 * alpha, 2.0 * alpha),
            1.0 * alpha,
            2.0 * alpha
        )

        params.update({
            "time_min": time_min_range,
            "time_max": time_max_range,
            "distribution": chosen_dist,
            "distribution_params": {"sigma": sigma}
        })
    elif shape_name == "ellipse":
        center_freq = random.uniform(random_freq_min, random_freq_max)
        center_time = random.uniform(random_time_min, random_time_max)
        radius_freq = random.uniform(10, (random_freq_max - random_freq_min) * alpha)
        radius_time = random.uniform(min_float_value, duration * alpha)

        params.update({
            "center_freq": clamp(center_freq, 0, sr),
            "center_time": clamp(center_time, 0, duration),
            "radius_freq": clamp(radius_freq, 10, sr),
            "radius_time": clamp(radius_time, min_float_value, duration)
        })
    else:
        # Default handling for shapes like circle
        center_freq = random.uniform(random_freq_min, random_freq_max)
        center_time = random.uniform(random_time_min, random_time_max)
        radius_freq = random.uniform(10, (random_freq_max - random_freq_min) * alpha)
        radius_time = random.uniform(min_float_value, duration * alpha)

        params.update({
            "center_freq": clamp(center_freq, 0, sr),
            "center_time": clamp(center_time, 0, duration),
            "radius_freq": clamp(radius_freq, 10, sr),
            "radius_time": clamp(radius_time, min_float_value, duration)
        })

    return params

# =========================================================
# 9) Create Random Noise Pipeline (Fixed Shapes and Patterns)
# =========================================================
def create_random_noise_pipeline(
    spectro_mod,
    max_shapes=5,
    max_patterns=3,
    apply_blur=False,
    blur_sigma=1.0,
    duration=8.0,
    sr=16000,
    freq_min=20,
    min_float_value=0.001,
    alpha=1.0,
    ratio_shape=None,
    ratio_pattern=None,
    max_db_power=20,
    min_db_power=10
):
    """
    ratio_shape, ratio_pattern example:
    ratio_shape = {"circle": 1, "rectangle": 2, "ellipse": 3, ...}
    ratio_pattern = {"linear": 1, "random": 2, "convex": 1, ...}
    """
    pipeline = NoisePipeline(
        spectro_mod,
        apply_blur=apply_blur,
        blur_sigma=blur_sigma
    )

    # Default candidates if ratios are None
    default_shapes = [
        "circle", "rectangle", "ellipse", "horizontal_spike", "vertical_spike",
        "fog", "pillar", "horizontal_line", "vertical_line",
        "horizontal_range_dist_db", "vertical_range_dist_db",
        "trapezoid", "hill", "wave_pattern", "polygon"
    ]
    default_patterns = [
        "linear", "random", "n_linear_repeat_t_sleep", "convex"
    ]

    if ratio_shape is None:
        ratio_shape = {s: 1 for s in default_shapes}
    if ratio_pattern is None:
        ratio_pattern = {p: 1 for p in default_patterns}

    shape_factory = pipeline.shape_factory
    pattern_factory = pipeline.pattern_factory

    # Add Shapes
    for _ in range(max_shapes):
        shape_name = pick_item_from_ratio(ratio_shape)
        try:
            shape_params = generate_shape_params(
                shape_name=shape_name,
                duration=duration,
                sr=sr,
                freq_min=freq_min,
                time_min=0.0,
                min_float_value=min_float_value,
                alpha=alpha,
                max_db_power=max_db_power,
                min_db_power=min_db_power
            )
            shape = shape_factory.create(shape_name, **shape_params)
            pipeline.add_shape(shape)
            print(f"Added Shape: {shape_name} -> {shape}")
        except Exception as e:
            print(f"Error configuring shape '{shape_name}': {e}.")

    # Add Patterns
    for _ in range(max_patterns):
        pattern_name = pick_item_from_ratio(ratio_pattern)
        try:
            # Select suitable shape names for the pattern
            if pattern_name == "n_linear_repeat_t_sleep":
                possible_shape_names = [
                    s for s in ratio_shape.keys()
                    if not s.endswith('_range_dist_db')
                ]
            else:
                possible_shape_names = [
                    s for s in ratio_shape.keys()
                    if s.startswith('horizontal_') or
                    s.startswith('vertical_') or
                    s.endswith('_range_dist_db')
                ]

            if not possible_shape_names:
                print("No suitable shapes available for the selected pattern.")
                continue

            shape_name = pick_item_from_ratio(
                {s: ratio_shape[s] for s in possible_shape_names}
            )

            shape_params = generate_shape_params(
                shape_name=shape_name,
                duration=duration,
                sr=sr,
                freq_min=freq_min,
                time_min=0.0,
                min_float_value=min_float_value,
                alpha=alpha,
                max_db_power=max_db_power,
                min_db_power=min_db_power
            )

            # Determine direction based on shape name
            if shape_name.startswith('horizontal_') or shape_name.endswith('_range_dist_db'):
                direction = 'freq'
            elif shape_name.startswith('vertical_') or shape_name.endswith('_range_dist_db'):
                direction = 'time'
            else:
                direction = 'time'

            pattern_kwargs = {
                "shape_name": shape_name,
                "shape_params": shape_params
            }

            if pattern_name == "linear":
                pattern_kwargs.update({
                    "direction": direction,
                    "repeat": 3,  # Fixed number
                    "spacing": 1.0  # Fixed spacing
                })
            elif pattern_name == "random":
                pattern_kwargs.update({
                    "n": 5,  # Fixed number
                    "freq_range": (freq_min, sr),
                    "time_range": (0.0, duration)
                })
            elif pattern_name == "n_linear_repeat_t_sleep":
                pattern_kwargs.update({
                    "repeat": 3,  # Fixed number
                    "repeat_time": 0.5,  # Fixed repeat time
                    "sleep_time": 1.0,  # Fixed sleep time
                    "start_time": 0.0,  # Fixed start time
                    "direction": direction
                })
            elif pattern_name == "convex":
                pattern_kwargs.update({
                    "freq_min": freq_min,
                    "freq_max": sr,
                    "time_min": 0.0,
                    "time_max": duration,
                    "n": 5  # Fixed number
                })

            pattern = pattern_factory.create(pattern_name, pattern_kwargs)
            if pattern:
                pipeline.add_pattern(pattern)
                print(f"Added Pattern: {pattern_name} -> {pattern}")
        except Exception as e:
            print(f"Error configuring pattern '{pattern_name}': {e}.")

    return pipeline

# =========================================================
# 10) Spectrogram to Audio Reconstruction
# =========================================================
def reconstruct_with_griffinlim(spectrogram_db, n_iter=32, hop_length=512):
    amplitude = librosa.db_to_amplitude(spectrogram_db, ref=1.0)
    new_audio = librosa.griffinlim(
        amplitude,
        n_iter=n_iter,
        hop_length=hop_length,
        window='hann'
    )
    return new_audio

def reconstruct_audio_from_final_spectrogram(spectro_mod, hop_length=512):
    stft_orig = librosa.stft(
        spectro_mod.signal_with_noise,
        n_fft=spectro_mod.n_fft,
        hop_length=spectro_mod.hop_length,
        window='hann'
    )
    phase = np.angle(stft_orig)

    magnitude = librosa.db_to_amplitude(spectro_mod.S_db, ref=1.0)
    new_stft = magnitude * np.exp(1j * phase)

    new_audio = librosa.istft(
        new_stft,
        hop_length=hop_length,
        window='hann'
    )
    return new_audio

# =========================================================
# 11) Example Usage
# =========================================================
def main():
    # Parameters
    sr = 16000
    duration = 12.0
    n_samples = int(sr * duration)
    signal = np.random.normal(-40, 1, n_samples)  # Adjusted mean

    # Randomly select SpectrogramModifier parameters
    noise_types = ['normal', 'uniform', 'perlin']
    noise_type = random.choice(noise_types)
    noise_strength = random.uniform(5, 15)
    noise_params = {}
    if noise_type == 'normal':
        noise_params = {'mean': 0.0, 'std': 1.0}
    elif noise_type == 'uniform':
        noise_params = {'low': -1.0, 'high': 1.0}
    elif noise_type == 'perlin':
        noise_params = {'seed': random.randint(0, 1000), 'scale': random.uniform(20.0, 100.0)}

    spectro_mod = SpectrogramModifier(
        sample_rate=sr,
        noise_strength=noise_strength,
        noise_type=noise_type,
        noise_params=noise_params
    )

    # Define shape and pattern ratios
    ratio_shape = {
        "circle": 0,
        "rectangle": 0,
        "ellipse": 0,
        "horizontal_spike": 1,
        "vertical_spike": 1,
        "fog": 0,
        "pillar": 0,
        "horizontal_line": 1,
        "vertical_line": 1,
        "horizontal_range_dist_db": 1,
        "vertical_range_dist_db": 1,
        "trapezoid": 0,
        "hill": 0,
        "wave_pattern": 0,
        "polygon": 0
    }

    ratio_pattern = {
        "linear": 0,
        "random": 0,
        "n_linear_repeat_t_sleep": 1,
        "convex": 0
    }

    # Create NoisePipeline with fixed number of shapes and patterns
    pipeline = create_random_noise_pipeline(
        spectro_mod,
        max_shapes=10,   # Fixed number of shapes
        max_patterns=2,  # Fixed number of patterns
        apply_blur=True,
        blur_sigma=1.0,
        duration=duration,
        sr=sr / 2,  # As per instruction
        freq_min=0,
        min_float_value=0.001,
        alpha=1.0,
        ratio_shape=ratio_shape,
        ratio_pattern=ratio_pattern,
        max_db_power=40,
        min_db_power=20
    )

    # Generate the final spectrogram with noise
    result_db = pipeline.generate(signal)
    print("NoisePipeline applied successfully.")

    # Create output directories
    output_dirs = {
        "audio": "output/audio",
        "spectrogram_with_axes": "output/linear_spectrogram_with_axes",
        "spectrogram_no_axes": "output/linear_spectrogram_no_axes",
        "json": "output/json"
    }
    for dir_path in output_dirs.values():
        os.makedirs(dir_path, exist_ok=True)

    # Define file identifiers
    file_id = "000000001"
    audio_file_path = os.path.join(output_dirs["audio"], f"automated_audio_{file_id}.wav")
    spectrogram_with_axes_path = os.path.join(
        output_dirs["spectrogram_with_axes"],
        f"automated_linear_spectrogram_with_axes_{file_id}.png"
    )
    spectrogram_no_axes_path = os.path.join(
        output_dirs["spectrogram_no_axes"],
        f"automated_linear_spectrogram_no_axes_{file_id}.png"
    )
    json_file_path = os.path.join(output_dirs["json"], f"automated_audio_{file_id}.json")

    # Plot and save spectrogram with axes
    fig_with_axes, _ = spectro_mod.plot_spectrogram(
        show_labels=True,
        title='Final Spectrogram with Full Random'
    )
    fig_with_axes.savefig(spectrogram_with_axes_path)
    plt.show()
    plt.close(fig_with_axes)
    print(f"Spectrogram with axes saved to {spectrogram_with_axes_path}")

    # Plot and save spectrogram without axes
    fig_no_axes, _ = spectro_mod.plot_spectrogram(
        show_labels=False,
        title='Final Spectrogram without Axes'
    )
    fig_no_axes.savefig(spectrogram_no_axes_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig_no_axes)
    print(f"Spectrogram without axes saved to {spectrogram_no_axes_path}")

    # Reconstruct audio from the final spectrogram
    reconstructed = reconstruct_audio_from_final_spectrogram(spectro_mod, hop_length=512)

    # Save the reconstructed audio using soundfile
    sf.write(audio_file_path, reconstructed, int(sr))
    print(f"Reconstructed audio saved to {audio_file_path}")

    # Prepare JSON metadata
    extracted_data = {
        "file_path": audio_file_path,
        "file_name": os.path.basename(audio_file_path),
        "linear_spectrogram_with_axes": spectrogram_with_axes_path,
        "linear_spectrogram_no_axes": spectrogram_no_axes_path,
        "spectrogram_base": {
            "sample_rate": spectro_mod.sample_rate,
            "n_fft": spectro_mod.n_fft,
            "hop_length": spectro_mod.hop_length,
            "noise_strength": spectro_mod.noise_strength,
            "noise_type": spectro_mod.noise_type,
            "noise_params": spectro_mod.noise_params
        },
        "shapes": [
            {
                "type": shape.__class__.__name__,
                "parameters": shape.__dict__
            } for shape in pipeline.shapes
        ],
        "patterns": [
            {
                "type": pattern.__class__.__name__,
                "parameters": pattern.__dict__
            } for pattern in pipeline.patterns
        ],
    }

    # Save JSON metadata
    with open(json_file_path, 'w') as json_file:
        json.dump(extracted_data, json_file, indent=4)
    print(f"JSON metadata saved to {json_file_path}")

    # Optionally, play audio in Jupyter environment
    try:
        import IPython.display as ipd
        from IPython import get_ipython

        ipython = get_ipython()
        if ipython:
            audio_widget = ipd.Audio(data=reconstructed, rate=int(sr))
            display(audio_widget)
    except (ImportError, NameError):
        print(
            "Jupyter environment not detected. Audio has been saved as a WAV file."
        )

# =========================================================
# 11) Modified Example Usage for Batch Generation
# =========================================================
def batch_main(
    total_files=100000,
    batch_size=1000,  # Number of files per subdirectory
    output_base_dir="output",
    sr=16000,
    duration=12.0
):
    """
    Generates multiple audio files with modified spectrograms.

    Parameters:
    - total_files (int): Total number of files to generate.
    - batch_size (int): Number of files per subdirectory.
    - output_base_dir (str): Base directory for all outputs.
    - sr (int): Sample rate.
    - duration (float): Duration of each audio file in seconds.
    """
    # Ensure necessary output subdirectories exist
    output_dirs = {
        "audio": os.path.join(output_base_dir, "audio"),
        "spectrogram_with_axes": os.path.join(output_base_dir, "linear_spectrogram_with_axes"),
        "spectrogram_no_axes": os.path.join(output_base_dir, "linear_spectrogram_no_axes"),
        "json": os.path.join(output_base_dir, "json")
    }
    for dir_path in output_dirs.values():
        os.makedirs(dir_path, exist_ok=True)

    # Define shape and pattern ratios
    ratio_shape = {
        "circle": 0,
        "rectangle": 0,
        "ellipse": 0,
        "horizontal_spike": 1,
        "vertical_spike": 1,
        "fog": 0,
        "pillar": 0,
        "horizontal_line": 1,
        "vertical_line": 1,
        "horizontal_range_dist_db": 1,
        "vertical_range_dist_db": 1,
        "trapezoid": 0,
        "hill": 0,
        "wave_pattern": 0,
        "polygon": 0
    }

    ratio_pattern = {
        "linear": 0,
        "random": 0,
        "n_linear_repeat_t_sleep": 1,
        "convex": 0
    }

    shape_factory = ShapeFactory()
    pattern_factory = PatternFactory()

    # Loop over the total number of files
    for i in tqdm(range(1, total_files + 1), desc="Generating Files"):
        # Generate a zero-padded file_id (e.g., '000000001' to '100000')
        file_id = f"{i:09d}"

        # Parameters for noise generation
        noise_types = ['normal', 'uniform', 'perlin']
        noise_type = random.choice(noise_types)
        noise_strength = random.uniform(5, 15)
        noise_params = {}
        if noise_type == 'normal':
            noise_params = {'mean': 0.0, 'std': 1.0}
        elif noise_type == 'uniform':
            noise_params = {'low': -1.0, 'high': 1.0}
        elif noise_type == 'perlin':
            noise_params = {'seed': random.randint(0, 1000), 'scale': random.uniform(20.0, 100.0)}

        spectro_mod = SpectrogramModifier(
            sample_rate=sr,
            noise_strength=noise_strength,
            noise_type=noise_type,
            noise_params=noise_params
        )

        # Create NoisePipeline
        pipeline = create_random_noise_pipeline(
            spectro_mod,
            max_shapes=10,   # Fixed number of shapes
            max_patterns=2,  # Fixed number of patterns
            apply_blur=True,
            blur_sigma=1.0,
            duration=duration,
            sr=sr / 2,  # As per instruction
            freq_min=0,
            min_float_value=0.001,
            alpha=1.0,
            ratio_shape=ratio_shape,
            ratio_pattern=ratio_pattern,
            max_db_power=40,
            min_db_power=20
        )

        # Generate the final spectrogram with noise
        try:
            result_db = pipeline.generate(np.random.normal(-40, 1, int(sr * duration)))
        except Exception as e:
            print(f"Error generating spectrogram for file_id {file_id}: {e}")
            continue

        # Define file paths
        audio_file_path = os.path.join(output_dirs["audio"], f"automated_audio_{file_id}.wav")
        spectrogram_with_axes_path = os.path.join(
            output_dirs["spectrogram_with_axes"],
            f"automated_linear_spectrogram_with_axes_{file_id}.png"
        )
        spectrogram_no_axes_path = os.path.join(
            output_dirs["spectrogram_no_axes"],
            f"automated_linear_spectrogram_no_axes_{file_id}.png"
        )
        json_file_path = os.path.join(output_dirs["json"], f"automated_audio_{file_id}.json")

        # Plot and save spectrogram with axes
        try:
            fig_with_axes, _ = spectro_mod.plot_spectrogram(
                show_labels=True,
                title='Final Spectrogram with Full Random'
            )
            fig_with_axes.savefig(spectrogram_with_axes_path)
            plt.close(fig_with_axes)
        except Exception as e:
            print(f"Error saving spectrogram with axes for file_id {file_id}: {e}")

        # Plot and save spectrogram without axes
        try:
            fig_no_axes, _ = spectro_mod.plot_spectrogram(
                show_labels=False,
                title='Final Spectrogram without Axes'
            )
            fig_no_axes.savefig(spectrogram_no_axes_path, bbox_inches='tight', pad_inches=0)
            plt.close(fig_no_axes)
        except Exception as e:
            print(f"Error saving spectrogram without axes for file_id {file_id}: {e}")

        # Reconstruct audio from the final spectrogram
        try:
            reconstructed = reconstruct_audio_from_final_spectrogram(spectro_mod, hop_length=512)
            # Save the reconstructed audio using soundfile
            sf.write(audio_file_path, reconstructed, int(sr))
        except Exception as e:
            print(f"Error reconstructing/saving audio for file_id {file_id}: {e}")

        # Prepare JSON metadata
        try:
            extracted_data = {
                "file_path": audio_file_path,
                "file_name": os.path.basename(audio_file_path),
                "spectrogram_with_axes": spectrogram_with_axes_path,
                "spectrogram_no_axes": spectrogram_no_axes_path,
                "spectrogram_base": {
                    "sample_rate": spectro_mod.sample_rate,
                    "n_fft": spectro_mod.n_fft,
                    "hop_length": spectro_mod.hop_length,
                    "noise_strength": spectro_mod.noise_strength,
                    "noise_type": spectro_mod.noise_type,
                    "noise_params": spectro_mod.noise_params
                },
                "shapes": [
                    {
                        "type": shape.__class__.__name__,
                        "parameters": shape.__dict__
                    } for shape in pipeline.shapes
                ],
                "patterns": [
                    {
                        "type": pattern.__class__.__name__,
                        "parameters": pattern.__dict__
                    } for pattern in pipeline.patterns
                ],
            }

            # Save JSON metadata
            with open(json_file_path, 'w') as json_file:
                json.dump(extracted_data, json_file, indent=4)
        except Exception as e:
            print(f"Error saving JSON metadata for file_id {file_id}: {e}")

    print("Batch generation completed successfully.")

if __name__ == "__main__":
    batch_main(
        total_files=10,      # Total number of files to generate
        batch_size=1,         # Not used in this script but can be utilized for advanced batching
        output_base_dir="output",
        sr=16000,
        duration=12.0
    )


Generating Files:   0%|          | 0/10 [00:00<?, ?it/s]

Added Shape: vertical_spike -> VerticalSpikeDBShape({'center_freq': 6.353600849081804, 'center_time': 3.745142279664994, 'radius_freq': 3855.380180296666, 'radius_time': 0.4842988438110288, 'strength_dB': 24.420876806623987})
Added Shape: vertical_range_dist_db -> VerticalRangeDistributionDBShape({'time_min': 9.446503245619828, 'time_max': 11.119448492194296, 'strength_dB': 30.825591163898697, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1.974133234699117}})
Added Shape: vertical_line -> VerticalLineDBShape({'center_time': 7.308702364090409, 'strength_dB': 20.677255761303805, 'thickness': 2})
Added Shape: horizontal_line -> HorizontalLineDBShape({'center_freq': 4574.883949573692, 'strength_dB': 37.0122071364885, 'thickness': 2})
Added Shape: horizontal_spike -> HorizontalSpikeDBShape({'center_freq': 745.604782089431, 'center_time': 6.848211088649393, 'radius_freq': 43.445004169672586, 'radius_time': 0.2080909786437232, 'strength_dB': 34.720003654256196})
Added Shape: ho

Generating Files:  10%|█         | 1/10 [00:00<00:08,  1.03it/s]

Added Shape: vertical_line -> VerticalLineDBShape({'center_time': 1.4112612214727922, 'strength_dB': 27.47546984193887, 'thickness': 1})
Added Shape: vertical_range_dist_db -> VerticalRangeDistributionDBShape({'time_min': 8.603166598948473, 'time_max': 10.47148760181229, 'strength_dB': 28.803630885309275, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1.7073751715511132}})
Added Shape: horizontal_line -> HorizontalLineDBShape({'center_freq': 6276.655046521275, 'strength_dB': 29.785452516544673, 'thickness': 3})
Added Shape: vertical_range_dist_db -> VerticalRangeDistributionDBShape({'time_min': 4.373130849855699, 'time_max': 11.04816924041719, 'strength_dB': 28.116395720156454, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1.4551124914010671}})
Added Shape: horizontal_spike -> HorizontalSpikeDBShape({'center_freq': 1683.9645775984645, 'center_time': 6.945789270745976, 'radius_freq': 91.64330081811431, 'radius_time': 0.6479882851700168, 'strength_dB': 35.693

Generating Files:  20%|██        | 2/10 [00:01<00:04,  1.70it/s]

...

Added Shape: horizontal_range_dist_db -> HorizontalRangeDistributionDBShape({'freq_min': 188.94901686480713, 'freq_max': 189.14901686480712, 'strength_dB': 20.168803043355357, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1085.9272760874119}})
Added Shape: horizontal_spike -> HorizontalSpikeDBShape({'center_freq': 385.0785908152532, 'center_time': 2.414627784848977, 'radius_freq': 62.74626112619735, 'radius_time': 0.32308557113351516, 'strength_dB': 39.79763300611968})
Added Shape: horizontal_line -> HorizontalLineDBShape({'center_freq': 5675.883409031359, 'strength_dB': 34.41181357090355, 'thickness': 2})
Added Shape: horizontal_range_dist_db -> HorizontalRangeDistributionDBShape({'freq_min': 95.17828930906069, 'freq_max': 95.37828930906068, 'strength_dB': 28.16638393695918, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1018.1135916146425}})
Added Shape: vertical_spike -> VerticalSpikeDBShape({'center_freq': 3.2544585068664023, 'center_time': 2.6958669074

Generating Files:  90%|█████████ | 9/10 [00:03<00:00,  2.95it/s]

Added Shape: horizontal_spike -> HorizontalSpikeDBShape({'center_freq': 2665.0051611344884, 'center_time': 1.9783659352076117, 'radius_freq': 36.27716647308848, 'radius_time': 0.13431988942041498, 'strength_dB': 35.00752964154505})
Added Shape: vertical_line -> VerticalLineDBShape({'center_time': 8.643132456117824, 'strength_dB': 38.70712031096193, 'thickness': 2})
Added Shape: horizontal_range_dist_db -> HorizontalRangeDistributionDBShape({'freq_min': 2571.105708547999, 'freq_max': 2571.3057085479986, 'strength_dB': 39.67055278339815, 'distribution': 'gaussian', 'distribution_params': {'sigma': 1044.7129997528812}})
Added Shape: horizontal_spike -> HorizontalSpikeDBShape({'center_freq': 5707.301344176246, 'center_time': 8.51521246314007, 'radius_freq': 30.314937110112908, 'radius_time': 0.05, 'strength_dB': 30.633992823460773})
Added Shape: vertical_range_dist_db -> VerticalRangeDistributionDBShape({'time_min': 5.0267577024202215, 'time_max': 11.660002777980596, 'strength_dB': 28.6860

Generating Files: 100%|██████████| 10/10 [00:03<00:00,  2.54it/s]

Batch generation completed successfully.
